# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
#instalar o webdriver compatível com o meu navegador
#pip install webdriver-manager
#from webdriver_manager.edge import EdgeDriverManager
# selenium 3
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager

driver = webdriver.Edge(EdgeChromiumDriverManager().install())
# selenium 4
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager

driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))

In [ ]:
#pip install selenium
from selenium import webdriver #Em vez de importar a bib inteira, só pegamos o webdriver
from selenium.webdriver.common.keys import Keys
#pip install cx-Freeze #para transformar em um exe.


# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = webdriver.Chrome(options=chrome_options)

browser = webdriver.Edge()
# Passo 1: Pegar a cotação do dólar
# entrar no google
browser.get("https://www.google.com/")
browser.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação dólar", Keys.ENTER)
#browser.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[4]/center/input[1]").click()
dollarValue = browser.find_element("xpath",'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(dollarValue)

# Passo 2: Pegar a cotação do euro
browser.get("https://www.google.com/")
browser.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação euro", Keys.ENTER)
euroValue = browser.find_element("xpath",'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(euroValue)

# Passo 3: Pegar a cotação do ouro
browser.get("https://www.melhorcambio.com/ouro-hoje")
goldValue = browser.find_element("xpath", '//*[@id="comercial"]').get_attribute("value")
goldValue =  goldValue.replace(",", ".")
print(goldValue)


with open(r"C:\Users\felip\OneDrive\Documentos\cotas.txt", "w") as file:
    file.write(f"O preço do dólar é: {dollarValue}\nO preço do ouro é: {goldValue}\nO preço do euro é: {euroValue}")
    
browser.quit()




### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
# Passo 4: Importar a base de dados e Atualizar a base
import pandas as pd

table = pd.read_excel("Produtos.xlsx")
print(table)

- Atualizando os preços e o cálculo do Preço Final

In [ ]:
import pandas as pd

table = pd.read_excel("Produtos.xlsx")
# Passo 5: Recalcular os preços
# atualizar cotação
# cotação dólar
table.loc[table["Moeda"] == "Dólar", "Cotação"] = float(dollarValue)
table.loc[table["Moeda"] == "Euro", "Cotação"] = float(euroValue)
table.loc[table["Moeda"] == "Ouro", "Cotação"] = float(goldValue)
# preço de compra = cotação * preço original
table["Preço de Compra"] = table["Cotação"] * table["Preço Original"]

# preço de venda = preço de compra * margem
#table["Preço de Venda"] = table["Preço de Compra"] * table["Margem"]
# formatado
table["Preço de Venda"] = table["Preço de Venda"].map("R${:.2f}".format)#vai formatar os valores.
print(table)


### Agora vamos exportar a nova base de preços atualizada

In [ ]:
# Passo 6: Exportar a base atualizada
table.to_excel("ProdutosNew.xlsx", index = False)# index false vai tirar index
